In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import time
from numba import njit,guvectorize,float64
import scipy.optimize as opt
from matplotlib import pyplot as plt

#Set
t = np.arange(1, 101)
NT = len(t)

In [2]:
#Parameters
fosslim = 6000 # Maximum cumulative extraction fossil fuels (GtC); denoted by CCum
tstep  = 5 # Years per Period
ifopt  = 0 # Indicator where optimized is 1 and base is 0

In [3]:
#Preferences

elasmu = 1.45 #  Elasticity of marginal utility of consumption
prstp = 0.015 #   Initial rate of social time preference per year 

#** Population and technology
gama  = 0.300 #   Capital elasticity in production function         /.300 /
pop0  = 7403   # Initial world population 2015 (millions)          /7403 /
popadj = 0.134 #  Growth rate to calibrate to 2050 pop projection  /0.134/
popasym = 11500 # Asymptotic population (millions)                 /11500/
dk  = 0.100 #     Depreciation rate on capital (per year)           /.100 /
q0  = 105.5 #     Initial world gross output 2015 (trill 2010 USD) /105.5/
k0  = 223 #     Initial capital value 2015 (trill 2010 USD)        /223  /
a0  = 5.115 #     Initial level of total factor productivity       /5.115/
ga0  = 0.076 #    Initial growth rate for TFP per 5 years          /0.076/
dela  = 0.005 #   Decline rate of TFP per 5 years                  /0.005/

#** Emissions parameters
gsigma1  = -0.0152 # Initial growth of sigma (per year)            /-0.0152/
dsig  = -0.001 #   Decline rate of decarbonization (per period)    /-0.001 /
eland0 = 2.6 #  Carbon emissions from land 2015 (GtCO2 per year)   / 2.6   /
deland = 0.115 # Decline rate of land emissions (per period)        / .115  /
e0 = 35.85 #    Industrial emissions 2015 (GtCO2 per year)       /35.85  /
miu0  = 0.03 #   Initial emissions control rate for base case 2015  /.03    /

#** Carbon cycle
#* Initial Conditions
mat0 = 851 #  Initial Concentration in atmosphere 2015 (GtC)       /851  /
mu0  = 460 #  Initial Concentration in upper strata 2015 (GtC)     /460  /
ml0  = 1740 #  Initial Concentration in lower strata 2015 (GtC)    /1740 /
mateq = 588 # mateq Equilibrium concentration atmosphere  (GtC)    /588  /
mueq  = 360 # mueq Equilibrium concentration in upper strata (GtC) /360  /
mleq = 1720 # mleq Equilibrium concentration in lower strata (GtC) /1720 /

#* Flow paramaters, denoted by Phi_ij in the model
b12  = 0.12 #    Carbon cycle transition matrix                     /.12  /
b23  = 0.007 #   Carbon cycle transition matrix                    /0.007/
#* These are for declaration and are defined later
b11  = None   # Carbon cycle transition matrix
b21  = None  # Carbon cycle transition matrix
b22  = None  # Carbon cycle transition matrix
b32  = None  # Carbon cycle transition matrix
b33  = None  # Carbon cycle transition matrix
sig0  = None  # Carbon intensity 2010 (kgCO2 per output 2005 USD 2010)

#** Climate model parameters
t2xco2  = 3.1 # Equilibrium temp impact (oC per doubling CO2)    / 3.1 /
fex0  = 0.5 #   2015 forcings of non-CO2 GHG (Wm-2)              / 0.5 /
fex1  = 1.0 #   2100 forcings of non-CO2 GHG (Wm-2)              / 1.0 /
tocean0  = 0.0068 # Initial lower stratum temp change (C from 1900) /.0068/
tatm0  = 0.85 #  Initial atmospheric temp change (C from 1900)    /0.85/
c1  = 0.1005 #     Climate equation coefficient for upper level  /0.1005/
c3  = 0.088 #     Transfer coefficient upper to lower stratum    /0.088/
c4  = 0.025 #     Transfer coefficient for lower level           /0.025/
fco22x  = 3.6813 # eta in the model; Eq.22 : Forcings of equilibrium CO2 doubling (Wm-2)   /3.6813 /

#** Climate damage parameters
a10  = 0 #     Initial damage intercept                         /0   /
a20  = None #     Initial damage quadratic term
a1  = 0 #      Damage intercept                                 /0   /
a2  = 0.00236 #      Damage quadratic term                     /0.00236/
a3  = 2.00 #      Damage exponent                              /2.00   /

#** Abatement cost
expcost2 = 2.6 # Theta2 in the model, Eq. 10 Exponent of control cost function             / 2.6  /
pback  = 550 #   Cost of backstop 2010$ per tCO2 2015          / 550  /
gback  = 0.025 #   Initial cost decline backstop cost per period / .025/
limmiu  = 1.2 #  Upper limit on control rate after 2150        / 1.2 /
tnopol  = 45 #  Period before which no emissions controls base  / 45   /
cprice0  = 2 # Initial base carbon price (2010$ per tCO2)      / 2    /
gcprice  = 0.02 # Growth rate of base carbon price per year     /.02  /

#** Scaling and inessential parameters
#* Note that these are unnecessary for the calculations
#* They ensure that MU of first period's consumption =1 and PV cons = PV utilty
scale1  = 0.0302455265681763 #    Multiplicative scaling coefficient           /0.0302455265681763 /
scale2  = -10993.704 #    Additive scaling coefficient       /-10993.704/;

In [4]:
#* Parameters for long-run consistency of carbon cycle 
#(Question)
b11 = 1 - b12
b21 = b12*mateq/mueq
b22 = 1 - b21 - b23
b32 = b23*mueq/mleq
b33 = 1 - b32

#* Further definitions of parameters
a20 = a2
sig0 = e0/(q0*(1-miu0)) #From Eq. 14
lam = fco22x/ t2xco2 #From Eq. 25

l = np.zeros(NT)
l[0] = pop0 #Labor force
al = np.zeros(NT) 
al[0] = a0
gsig = np.zeros(NT) 
gsig[0] = gsigma1
sigma = np.zeros(NT)
sigma[0]= sig0
ga = ga0 * np.exp(-dela*5*(t-1)) #TFP growth rate dynamics, Eq. 7
pbacktime = pback * (1-gback)**(t-1) #Backstop price
etree = eland0*(1-deland)**(t-1) #Emissions from deforestration
rr = 1/((1+prstp)**(tstep*(t-1))) #Eq. 3
#The following three equations define the exogenous radiative forcing; used in Eq. 23  
forcoth = np.full(NT,fex0)
forcoth[0:18] = forcoth[0:18] + (1/17)*(fex1-fex0)*(t[0:18]-1)
forcoth[18:NT] = forcoth[18:NT] + (fex1-fex0)
optlrsav = (dk + .004)/(dk + .004*elasmu + prstp)*gama #Optimal long-run savings rate used for transversality (Question)
cost1 = np.zeros(NT)
cumetree = np.zeros(NT)
cumetree[0] = 100
cpricebase = cprice0*(1+gcprice)**(5*(t-1)) 

In [5]:
@njit('(float64[:], int32)')
def InitializeLabor(il,iNT):
    for i in range(1,iNT):
        il[i] = il[i-1]*(popasym / il[i-1])**popadj

@njit('(float64[:], int32)')        
def InitializeTFP(ial,iNT):
    for i in range(1,iNT):
        ial[i] = ial[i-1]/(1-ga[i-1])
        
@njit('(float64[:], int32)')        
def InitializeGrowthSigma(igsig,iNT):
    for i in range(1,iNT):
        igsig[i] = igsig[i-1]*((1+dsig)**tstep)
        
@njit('(float64[:], float64[:],float64[:],int32)')        
def InitializeSigma(isigma,igsig,icost1,iNT):
    for i in range(1,iNT):
        isigma[i] =  isigma[i-1] * np.exp(igsig[i-1] * tstep)
        icost1[i] = pbacktime[i] * isigma[i]  / expcost2 /1000
        
@njit('(float64[:], int32)')        
def InitializeCarbonTree(icumetree,iNT):
    for i in range(1,iNT):
        icumetree[i] = icumetree[i-1] + etree[i-1]*(5/3.666)


In [6]:
"""
Functions of the model
"""

"""
First: Functions related to emissions of carbon and weather damages
"""

# Retuns the total carbon emissions; Eq. 18
@njit('float64(float64[:],int32)') 
def fE(iEIND,index):
    return iEIND[index] + etree[index]

#Eq.14: Determines the emission of carbon by industry EIND
@njit('float64(float64[:],float64[:],float64[:],int32)') 
def fEIND(iYGROSS, iMIU, isigma,index):
    return isigma[index] * iYGROSS[index] * (1 - iMIU[index])

#Cumulative industrial emission of carbon
@njit('float64(float64[:],float64[:],int32)') 
def fCCA(iCCA,iEIND,index):
    return iCCA[index-1] + iEIND[index-1] * 5 / 3.666

#Cumulative total carbon emission
@njit('float64(float64[:],float64[:],int32)')
def fCCATOT(iCCA,icumetree,index):
    return iCCA[index] + icumetree[index]

#Eq. 22: the dynamics of the radiative forcing
@njit('float64(float64[:],int32)')
def fFORC(iMAT,index):
    return fco22x * np.log(iMAT[index]/588.000)/np.log(2) + forcoth[index]

# Dynamics of Omega; Eq.9
@njit('float64(float64[:],int32)')
def fDAMFRAC(iTATM,index):
    return a1*iTATM[index] + a2*iTATM[index]**a3

#Calculate damages as a function of Gross industrial production; Eq.8 
@njit('float64(float64[:],float64[:],int32)')
def fDAMAGES(iYGROSS,iDAMFRAC,index):
    return iYGROSS[index] * iDAMFRAC[index]

#Dynamics of Lambda; Eq. 10 - cost of the reudction of carbon emission (Abatement cost)
@njit('float64(float64[:],float64[:],float64[:],int32)') 
def fABATECOST(iYGROSS,iMIU,icost1,index):
    return iYGROSS[index] * icost1[index] * iMIU[index]**expcost2

#Marginal Abatement cost
@njit('float64(float64[:],int32)')
def fMCABATE(iMIU,index):
    return pbacktime[index] * iMIU[index]**(expcost2-1)

#Price of carbon reduction
@njit('float64(float64[:],int32)')
def fCPRICE(iMIU,index):
    return pbacktime[index] * (iMIU[index])**(expcost2-1)

#Eq. 19: Dynamics of the carbon concentration in the atmosphere
@njit('float64(float64[:],float64[:],float64[:],int32)') 
def fMAT(iMAT,iMU,iE,index):
    if(index == 0):
        return mat0
    else:
        return iMAT[index-1]*b11 + iMU[index-1]*b21 + iE[index-1] * 5 / 3.666

#Eq. 21: Dynamics of the carbon concentration in the ocean LOW level
@njit('float64(float64[:],float64[:],int32)') 
def fML(iML,iMU,index):
    if(index == 0):
        return ml0
    else:
        return iML[index-1] * b33  + iMU[index-1] * b23

#Eq. 20: Dynamics of the carbon concentration in the ocean UP level
@njit('float64(float64[:],float64[:],float64[:],int32)') 
def fMU(iMAT,iMU,iML,index):
    if(index == 0):
        return mu0
    else:
        return iMAT[index-1]*b12 + iMU[index-1]*b22 + iML[index-1]*b32

#Eq. 23: Dynamics of the atmospheric temperature
@njit('float64(float64[:],float64[:],float64[:],int32)') 
def fTATM(iTATM,iFORC,iTOCEAN,index):
    if(index == 0):
        return tatm0
    else:
        return iTATM[index-1] + c1 * (iFORC[index] - (fco22x/t2xco2) * iTATM[index-1] - c3 * (iTATM[index-1] - iTOCEAN[index-1]))

#Eq. 24: Dynamics of the ocean temperature
@njit('float64(float64[:],float64[:],int32)')
def fTOCEAN(iTATM,iTOCEAN,index):
    if(index == 0):
        return tocean0
    else:
        return iTOCEAN[index-1] + c4 * (iTATM[index-1] - iTOCEAN[index-1])

"""
Second: Function related to economic variables
"""

#The total production without climate losses denoted previously by YGROSS
@njit('float64(float64[:],float64[:],float64[:],int32)')
def fYGROSS(ial,il,iK,index):
    return ial[index] * ((il[index]/1000)**(1-gama)) * iK[index]**gama

#The production under the climate damages cost
@njit('float64(float64[:],float64[:],int32)')
def fYNET(iYGROSS, iDAMFRAC, index):
    return iYGROSS[index] * (1 - iDAMFRAC[index])

#Production after abatement cost
@njit('float64(float64[:],float64[:],int32)')
def fY(iYNET,iABATECOST,index):
    return iYNET[index] - iABATECOST[index]

#Consumption Eq. 11
@njit('float64(float64[:],float64[:],int32)')
def fC(iY,iI,index):
    return iY[index] - iI[index]

#Per capita consumption, Eq. 12
@njit('float64(float64[:],float64[:],int32)')
def fCPC(iC,il,index):
    return 1000 * iC[index] / il[index]

#Saving policy: investment
@njit('float64(float64[:],float64[:],int32)')
def fI(iS,iY,index):
    return iS[index] * iY[index] 

#Capital dynamics Eq. 13
@njit('float64(float64[:],float64[:],int32)')
def fK(iK,iI,index):
    if(index == 0):
        return k0
    else:
        return (1-dk)**tstep * iK[index-1] + tstep * iI[index-1]

#Interest rate equation; Eq. 26 added in personal notes
@njit('float64(float64[:],int32)')
def fRI(iCPC,index):
    return (1 + prstp) * (iCPC[index+1]/iCPC[index])**(elasmu/tstep) - 1

#Periodic utility: A form of Eq. 2
@njit('float64(float64[:],float64[:],int32)')
def fCEMUTOTPER(iPERIODU,il,index):
    return iPERIODU[index] * il[index] * rr[index]

#The term between brackets in Eq. 2
@njit('float64(float64[:],float64[:],int32)')
def fPERIODU(iC,il,index):
    return ((iC[index]*1000/il[index])**(1-elasmu) - 1) / (1 - elasmu) - 1

#utility function
@guvectorize([(float64[:], float64[:])], '(n), (m)')
def fUTILITY(iCEMUTOTPER, resUtility):
    resUtility[0] = tstep * scale1 * np.sum(iCEMUTOTPER) + scale2


In [7]:

"""
In this part we implement the objective function
"""

# * Control rate limits
MIU_lo = np.full(NT,0.01)
MIU_up = np.full(NT,limmiu)
MIU_up[0:29] = 1
MIU_lo[0] = miu0
MIU_up[0] = miu0
MIU_lo[MIU_lo==MIU_up] = 0.99999*MIU_lo[MIU_lo==MIU_up]
bnds1=[]
for i in range(NT):
    bnds1.append((MIU_lo[i],MIU_up[i]))
# * Control variables
lag10 = t > NT - 10
S_lo = np.full(NT,1e-1)
S_lo[lag10] = optlrsav
S_up = np.full(NT,0.9)
S_up[lag10] = optlrsav
S_lo[S_lo==S_up] = 0.99999*S_lo[S_lo==S_up]
bnds2=[]
for i in range(NT):
    bnds2.append((S_lo[i],S_up[i]))
    
# Arbitrary starting values for the control variables:
S_start = np.full(NT,0.2)
S_start[S_start < S_lo] = S_lo[S_start < S_lo]
S_start[S_start > S_up] = S_lo[S_start > S_up]
MIU_start = 0.99*MIU_up
MIU_start[MIU_start < MIU_lo] = MIU_lo[MIU_start < MIU_lo]
MIU_start[MIU_start > MIU_up] = MIU_up[MIU_start > MIU_up]

K = np.zeros(NT)
YGROSS = np.zeros(NT)
EIND = np.zeros(NT)
E = np.zeros(NT)
CCA = np.zeros(NT)
CCATOT = np.zeros(NT)
MAT = np.zeros(NT)
ML = np.zeros(NT)
MU = np.zeros(NT)
FORC = np.zeros(NT)
TATM = np.zeros(NT)
TOCEAN = np.zeros(NT)
DAMFRAC = np.zeros(NT)
DAMAGES = np.zeros(NT)
ABATECOST = np.zeros(NT)
MCABATE = np.zeros(NT)
CPRICE = np.zeros(NT)
YNET = np.zeros(NT)
Y = np.zeros(NT)
I = np.zeros(NT)
C = np.zeros(NT)
CPC = np.zeros(NT)
RI = np.zeros(NT)
PERIODU = np.zeros(NT)
CEMUTOTPER = np.zeros(NT)

In [8]:
#The objective function
#It returns the utility as scalar
def fOBJ(x,sign,iI,iK,ial,il,iYGROSS,isigma,iEIND,iE,iCCA,iCCATOT,icumetree,iMAT,iMU,iML,iFORC,iTATM,iTOCEAN,iDAMFRAC,iDAMAGES,iABATECOST,icost1,iMCABATE,
         iCPRICE,iYNET,iY,iC,iCPC,iPERIODU,iCEMUTOTPER,iRI,iNT):
    
    iMIU = x[0:NT]
    iS = x[NT:(2*NT)]
    
    for i in range(iNT):
        iK[i] = fK(iK,iI,i)
        iYGROSS[i] = fYGROSS(ial,il,iK,i)
        iEIND[i] = fEIND(iYGROSS, iMIU, isigma,i)
        iE[i] = fE(iEIND,i)
        iCCA[i] = fCCA(iCCA,iEIND,i)
        iCCATOT[i] = fCCATOT(iCCA,icumetree,i)
        iMAT[i] = fMAT(iMAT,iMU,iE,i)
        iML[i] = fML(iML,iMU,i)
        iMU[i] = fMU(iMAT,iMU,iML,i)
        iFORC[i] = fFORC(iMAT,i)
        iTATM[i] = fTATM(iTATM,iFORC,iTOCEAN,i)
        iTOCEAN[i] = fTOCEAN(iTATM,iTOCEAN,i)
        iDAMFRAC[i] = fDAMFRAC(iTATM,i)
        iDAMAGES[i] = fDAMAGES(iYGROSS,iDAMFRAC,i)
        iABATECOST[i] = fABATECOST(iYGROSS,iMIU,icost1,i)
        iMCABATE[i] = fMCABATE(iMIU,i)
        iCPRICE[i] = fCPRICE(iMIU,i)
        iYNET[i] = fYNET(iYGROSS, iDAMFRAC, i)
        iY[i] = fY(iYNET,iABATECOST,i)
        iI[i] = fI(iS,iY,i)
        iC[i] = fC(iY,iI,i)
        iCPC[i] = fCPC(iC,il,i)
        iPERIODU[i] = fPERIODU(iC,il,i)
        iCEMUTOTPER[i] = fCEMUTOTPER(iPERIODU,il,i)
        iRI = fRI(iCPC,i)
        
    resUtility = np.zeros(1)
    fUTILITY(iCEMUTOTPER, resUtility)
    
    return sign*resUtility[0]

In [9]:
ACTIONS = ['M+', 'S+', 'M-', 'S-', 'M+S+', 'M+S-', 'M-S+', 'M-S-']     # available actions
EPSILON = 0.1   # greedy police
ALPHA = 0.1     # learning rate
GAMMA = 0.9    # discount factor
MAX_EPISODES = 2000   # maximum episodes
Qmax = 4500
n_state = 40

In [10]:
miu_scaled = 1/n_state
s_scaled = 0.9/n_state

def get_env_feedback(MIU, S, A):
    if A == 'M+':    
        if MIU == n_state-1:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU+1
            S_ = S
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0
                
    elif A == 'S+':    
        if S == n_state-1:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU
            S_ = S+1
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0
        
    elif A == 'M-':
        if MIU == 0:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU-1
            S_ = S
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0    
        
    elif A == 'S-':
        if S == 0:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU
            S_ = S-1
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0
            
    elif A == 'M+S+':
        if MIU == n_state-1 or S == n_state-1:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU+1
            S_ = S+1
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0
            
    elif A == 'M+S-':
        if MIU == n_state-1 or S == 0:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU+1
            S_ = S-1
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0
            
    elif A == 'M-S+':
        if MIU == 0 or S == n_state-1:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU-1
            S_ = S+1
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0
            
    elif A == 'M-S-':
        if MIU == 0 or S == 0:
            MIU_ = MIU
            S_ = S
            R = 0
        else:
            MIU_ = MIU-1
            S_ = S-1
            x_ = np.concatenate([np.full(NT, (MIU_+1)*miu_scaled),np.full(NT, (S_+1)*s_scaled)])
            Q = fOBJ(x_,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)
            if Q > Qmax:
                R=1
                MIU_ = 'terminal'
                S_ = 'terminal'
            else:
                R=0
            
    return MIU_, S_, R

In [11]:
InitializeLabor(l,NT)
InitializeTFP(al,NT)
InitializeGrowthSigma(gsig,NT)
InitializeSigma(sigma,gsig,cost1,NT)
InitializeCarbonTree(cumetree,NT)

In [12]:
q_table = np.zeros((n_state, n_state, len(ACTIONS)))
for episode in range(MAX_EPISODES):
    MIU = 0
    S = 0
    steps = 0
    is_terminated = False
    
    while not is_terminated:
        
        # greedy action
        if (np.random.uniform(low=0, high=1) < EPSILON) or (q_table[MIU][S].all() == 0):
            A = np.random.choice(len(ACTIONS)) 
        else:
            A = np.argmax(q_table[MIU][S])

        MIU_, S_, R = get_env_feedback(MIU, S, ACTIONS[A])  # take action & get next state and reward
        if MIU_ != 'terminal' and S_ != 'terminal':
            q_target = R + GAMMA * np.max(q_table[MIU_][S_])  # next state is not terminal
        else:
            q_target = R
            is_terminated = True
            
        q_table[MIU][S][A] = (1 - ALPHA) * q_table[MIU][S][A] + ALPHA * q_target
        
        MIU = MIU_
        S = S_  # move to next state
        
        steps += 1
        
    print("Episode {} completed in {} steps".format(episode + 1, steps))
    
print(q_table)

Episode 1 completed in 495 steps
Episode 2 completed in 776 steps
Episode 3 completed in 8304 steps
Episode 4 completed in 710 steps
Episode 5 completed in 2262 steps
Episode 6 completed in 206 steps
Episode 7 completed in 4838 steps
Episode 8 completed in 2977 steps
Episode 9 completed in 2686 steps
Episode 10 completed in 6826 steps
Episode 11 completed in 3822 steps
Episode 12 completed in 230 steps
Episode 13 completed in 3741 steps
Episode 14 completed in 113 steps
Episode 15 completed in 94 steps
Episode 16 completed in 126 steps
Episode 17 completed in 194 steps
Episode 18 completed in 1335 steps
Episode 19 completed in 212 steps
Episode 20 completed in 42 steps
Episode 21 completed in 40 steps
Episode 22 completed in 131 steps
Episode 23 completed in 33 steps
Episode 24 completed in 330 steps
Episode 25 completed in 49 steps
Episode 26 completed in 54 steps
Episode 27 completed in 30 steps
Episode 28 completed in 700 steps
Episode 29 completed in 38 steps
Episode 30 completed i

Episode 259 completed in 36 steps
Episode 260 completed in 48 steps
Episode 261 completed in 23 steps
Episode 262 completed in 21 steps
Episode 263 completed in 26 steps
Episode 264 completed in 20 steps
Episode 265 completed in 32 steps
Episode 266 completed in 26 steps
Episode 267 completed in 26 steps
Episode 268 completed in 24 steps
Episode 269 completed in 22 steps
Episode 270 completed in 31 steps
Episode 271 completed in 42 steps
Episode 272 completed in 41 steps
Episode 273 completed in 1823 steps
Episode 274 completed in 49 steps
Episode 275 completed in 25 steps
Episode 276 completed in 28 steps
Episode 277 completed in 28 steps
Episode 278 completed in 26 steps
Episode 279 completed in 53 steps
Episode 280 completed in 25 steps
Episode 281 completed in 44 steps
Episode 282 completed in 38 steps
Episode 283 completed in 20 steps
Episode 284 completed in 21 steps
Episode 285 completed in 23 steps
Episode 286 completed in 18 steps
Episode 287 completed in 37 steps
Episode 288 

Episode 514 completed in 25 steps
Episode 515 completed in 27 steps
Episode 516 completed in 28 steps
Episode 517 completed in 23 steps
Episode 518 completed in 34 steps
Episode 519 completed in 30 steps
Episode 520 completed in 21 steps
Episode 521 completed in 45 steps
Episode 522 completed in 31 steps
Episode 523 completed in 31 steps
Episode 524 completed in 25 steps
Episode 525 completed in 46 steps
Episode 526 completed in 20 steps
Episode 527 completed in 33 steps
Episode 528 completed in 23 steps
Episode 529 completed in 27 steps
Episode 530 completed in 29 steps
Episode 531 completed in 65 steps
Episode 532 completed in 19 steps
Episode 533 completed in 20 steps
Episode 534 completed in 27 steps
Episode 535 completed in 53 steps
Episode 536 completed in 33 steps
Episode 537 completed in 29 steps
Episode 538 completed in 20 steps
Episode 539 completed in 22 steps
Episode 540 completed in 41 steps
Episode 541 completed in 34 steps
Episode 542 completed in 26 steps
Episode 543 co

Episode 774 completed in 24 steps
Episode 775 completed in 31 steps
Episode 776 completed in 20 steps
Episode 777 completed in 27 steps
Episode 778 completed in 20 steps
Episode 779 completed in 23 steps
Episode 780 completed in 31 steps
Episode 781 completed in 31 steps
Episode 782 completed in 41 steps
Episode 783 completed in 36 steps
Episode 784 completed in 33 steps
Episode 785 completed in 20 steps
Episode 786 completed in 36 steps
Episode 787 completed in 76 steps
Episode 788 completed in 21 steps
Episode 789 completed in 25 steps
Episode 790 completed in 29 steps
Episode 791 completed in 24 steps
Episode 792 completed in 76 steps
Episode 793 completed in 22 steps
Episode 794 completed in 31 steps
Episode 795 completed in 34 steps
Episode 796 completed in 35 steps
Episode 797 completed in 21 steps
Episode 798 completed in 34 steps
Episode 799 completed in 48 steps
Episode 800 completed in 30 steps
Episode 801 completed in 26 steps
Episode 802 completed in 43 steps
Episode 803 co

Episode 1016 completed in 42 steps
Episode 1017 completed in 38 steps
Episode 1018 completed in 45 steps
Episode 1019 completed in 26 steps
Episode 1020 completed in 39 steps
Episode 1021 completed in 34 steps
Episode 1022 completed in 29 steps
Episode 1023 completed in 48 steps
Episode 1024 completed in 31 steps
Episode 1025 completed in 25 steps
Episode 1026 completed in 22 steps
Episode 1027 completed in 18 steps
Episode 1028 completed in 27 steps
Episode 1029 completed in 24 steps
Episode 1030 completed in 27 steps
Episode 1031 completed in 41 steps
Episode 1032 completed in 24 steps
Episode 1033 completed in 25 steps
Episode 1034 completed in 44 steps
Episode 1035 completed in 40 steps
Episode 1036 completed in 25 steps
Episode 1037 completed in 18 steps
Episode 1038 completed in 27 steps
Episode 1039 completed in 28 steps
Episode 1040 completed in 29 steps
Episode 1041 completed in 29 steps
Episode 1042 completed in 22 steps
Episode 1043 completed in 35 steps
Episode 1044 complet

Episode 1398 completed in 20 steps
Episode 1399 completed in 19 steps
Episode 1400 completed in 27 steps
Episode 1401 completed in 46 steps
Episode 1402 completed in 24 steps
Episode 1403 completed in 25 steps
Episode 1404 completed in 26 steps
Episode 1405 completed in 48 steps
Episode 1406 completed in 23 steps
Episode 1407 completed in 35 steps
Episode 1408 completed in 23 steps
Episode 1409 completed in 17 steps
Episode 1410 completed in 17 steps
Episode 1411 completed in 19 steps
Episode 1412 completed in 34 steps
Episode 1413 completed in 26 steps
Episode 1414 completed in 25 steps
Episode 1415 completed in 36 steps
Episode 1416 completed in 23 steps
Episode 1417 completed in 21 steps
Episode 1418 completed in 28 steps
Episode 1419 completed in 18 steps
Episode 1420 completed in 43 steps
Episode 1421 completed in 33 steps
Episode 1422 completed in 20 steps
Episode 1423 completed in 37 steps
Episode 1424 completed in 27 steps
Episode 1425 completed in 28 steps
Episode 1426 complet

Episode 1640 completed in 40 steps
Episode 1641 completed in 24 steps
Episode 1642 completed in 22 steps
Episode 1643 completed in 34 steps
Episode 1644 completed in 21 steps
Episode 1645 completed in 45 steps
Episode 1646 completed in 19 steps
Episode 1647 completed in 32 steps
Episode 1648 completed in 24 steps
Episode 1649 completed in 22 steps
Episode 1650 completed in 19 steps
Episode 1651 completed in 27 steps
Episode 1652 completed in 23 steps
Episode 1653 completed in 18 steps
Episode 1654 completed in 26 steps
Episode 1655 completed in 24 steps
Episode 1656 completed in 18 steps
Episode 1657 completed in 24 steps
Episode 1658 completed in 27 steps
Episode 1659 completed in 23 steps
Episode 1660 completed in 25 steps
Episode 1661 completed in 26 steps
Episode 1662 completed in 45 steps
Episode 1663 completed in 33 steps
Episode 1664 completed in 29 steps
Episode 1665 completed in 18 steps
Episode 1666 completed in 22 steps
Episode 1667 completed in 28 steps
Episode 1668 complet

Episode 1901 completed in 36 steps
Episode 1902 completed in 24 steps
Episode 1903 completed in 21 steps
Episode 1904 completed in 22 steps
Episode 1905 completed in 26 steps
Episode 1906 completed in 24 steps
Episode 1907 completed in 16 steps
Episode 1908 completed in 31 steps
Episode 1909 completed in 28 steps
Episode 1910 completed in 36 steps
Episode 1911 completed in 16 steps
Episode 1912 completed in 20 steps
Episode 1913 completed in 23 steps
Episode 1914 completed in 20 steps
Episode 1915 completed in 34 steps
Episode 1916 completed in 24 steps
Episode 1917 completed in 35 steps
Episode 1918 completed in 17 steps
Episode 1919 completed in 29 steps
Episode 1920 completed in 18 steps
Episode 1921 completed in 33 steps
Episode 1922 completed in 23 steps
Episode 1923 completed in 18 steps
Episode 1924 completed in 29 steps
Episode 1925 completed in 22 steps
Episode 1926 completed in 34 steps
Episode 1927 completed in 23 steps
Episode 1928 completed in 26 steps
Episode 1929 complet

In [47]:
MIU = 0
S = 0
steps = 0
is_terminated = False
    
while not is_terminated:
    A = np.argmax(q_table[MIU][S])
    MIU_, S_, R = get_env_feedback(MIU, S, ACTIONS[A]) 
    if MIU_ != 'terminal' and S_ != 'terminal':
        MIU = MIU_
        S = S_  # move to next state
    else:
        is_terminated = True  
    steps += 1

if ACTIONS[A] == 'M+':
    MIU += 1
elif ACTIONS[A] == 'S+':
    S +=1
elif ACTIONS[A] == 'M-':
    MIU -=1
elif ACTIONS[A] == 'S-':
    S -=1
elif ACTIONS[A] == 'M+S+':
    MIU += 1
    S += 1
elif ACTIONS[A] == 'M+S-':
    MIU += 1
    S-=1
elif ACTIONS[A] == 'M-S+':
    MIU -= 1
    S += 1
elif ACTIONS[A] == 'M-S-':
    MIU -= 1
    S -= 1

x = np.concatenate([np.full(NT, (MIU+1)*miu_scaled),np.full(NT, (S+1)*s_scaled)])
Q = fOBJ(x,1,I,K,al,l,YGROSS,sigma,EIND,E,CCA,CCATOT,cumetree,MAT,MU,ML,FORC,TATM,TOCEAN,DAMFRAC,DAMAGES,ABATECOST,cost1,MCABATE,
         CPRICE,YNET,Y,C,CPC,PERIODU,CEMUTOTPER,RI,NT)

print("Episode completed in {} steps with MIU: {} and S: {}, the target Q: {}".format(steps, (MIU+1)*miu_scaled, (S+1)*s_scaled, Q))

Episode completed in 15 steps with MIU: 0.4 and S: 0.2475, the target Q: 4500.093919409746
